In [41]:
import numpy as np
import re
import h5py

In [30]:
# preprocessing
to_process = ['assgn1/rt-polarity.neg', 'assgn1/rt-polarity.pos']

# n is size of embeddings
# iterate through the files
# for each file, iterate through rows,
# split on spaces (how to handle punctuation?)
def build_vocab(files):
    vocab = {"/s": 0, "pad":1}
    counter = 2
    for file_h in files:
        with open(file_h, "rb") as fh:
            for line in fh:
                # filters out punctuation (preserves contractions), not numbers
                words = re.findall(r"[\w']+", line)
                for word in words:
                    if word not in vocab:
                        vocab[word] = counter
                        counter += 1
    return vocab

vocab = build_vocab(to_process)

13525


In [31]:
def make_data(file_h, n):
    data = []
    with open(file_h, "rb") as fh:
        for line in fh:
            datum = []
            words = re.findall(r"[\w']+", line)
            for i in xrange(n):
                if i < len(words):
                    datum.append(vocab[words[i]])
                else:
                    datum.append(1)
            data.append(datum)
    return np.array(data)

In [32]:
pos = make_data(to_process[1], 10)
neg = make_data(to_process[0], 10)

In [42]:
print np.shape(pos), np.shape(neg)

(5331, 10) (5331, 10)


In [45]:
with h5py.File("movie_reviews.hdf5", "w") as f:
    dset1 = f.create_dataset("neg", data=neg)
    dset2 = f.create_dataset("pos", data=pos)
#    vocab = f.create_dataset("vocab", data=vocab)

In [ ]:
import numpy as np
import cPickle
from collections import defaultdict
import sys, re
import pandas as pd

def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, "rb") as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in xrange(vocab_size):
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)   
            if word in vocab:
               word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')  
            else:
                f.read(binary_len)
    return word_vecs

if __name__=="__main__":    
    w2v_file = sys.argv[1]     
    # Set vocab to a dict of word frequencies
    vocab = {}

    print "loading word2vec vectors...",
    w2v = load_bin_vec(w2v_file, vocab)
    print "word2vec loaded!"
    print "num words in word2vec: " + str(len(w2v))